# Simulation parameters
We create a parameter dataframe containing the simulations' parameters, and a convenience function to fill it.

In [1]:
import math
import pandas as pd

# create empty dataframe
columns = ["name",  # name of the parameter
           "info",  # additional info, if required
           "real_value",  # real value (i.e. not using arbitrary units)
           "real_range_max",  # max theoretical value (if meaningful)
           "real_range_min",  # min theoretical value (if meaningful)
           "real_um",  # unit of measure
           "sim_value",  # value in arbitrary units
           "sim_range_max",  # max value in arbitrary units
           "sim_range_min",  # min value in arbitrary units
           "sim_um",  # arbitrary units of measure
           "references",  # scientific reference for the parameter (if needed)
           "influence"]  # list of other parameters affected by the current parameter (not used)
g_parameters_df = pd.DataFrame(columns=columns)


# define convenience function to fill parameters dict
def fill_parameters(params: pd.DataFrame,
                    name=None,
                    info=None,
                    real_value=None,
                    real_range_max=None,
                    real_range_min=None,
                    real_um=None,
                    sim_value=None,
                    sim_range_max=None,
                    sim_range_min=None,
                    sim_um=None,
                    references=None,
                    influence=None):
    # get input arguments
    args = locals()

    # define dictionary of the new row
    new_parameter_dict = {}
    for column in columns:
        new_parameter_dict[column] = [args[column]]

    # add it to
    return pd.concat([params, pd.DataFrame(new_parameter_dict)])

Then we initialize the arbitrary units:

In [2]:
from pint import UnitRegistry

# define unit registry
ureg = UnitRegistry()

# define names
sau_name = "SpaceArbitraryUnit"
tau_name = "TimeArbitraryUnit"
afau_name = "AFsArbitraryUnit"

# define arbitrary units for space and time
ureg.define(f"{sau_name} = 800 * um = sau")
ureg.define(f"{tau_name} = 26 * min = tau")

# define arbitrary units
sim_s_au = 800 * ureg.um
sim_t_au = 26 * ureg.min
sim_af_au = 6000 * ureg("pg / mL")  # about the same of the max VEGF value reported by Na et al. (2003)
ureg.define(f"{afau_name} = {sim_af_au} = afau")

# add arbitrary units to dataframe
g_parameters_df = fill_parameters(g_parameters_df,
                                  name=sau_name,
                                  real_value=sim_s_au.magnitude,
                                  real_um=sim_s_au.units,
                                  sim_value=sim_s_au.to("sau").magnitude,
                                  sim_um=sim_s_au.to("sau").units)
g_parameters_df = fill_parameters(g_parameters_df,
                                  name=tau_name,
                                  real_value=sim_t_au.magnitude,
                                  real_um=sim_t_au.units,
                                  sim_value=sim_t_au.to("tau").magnitude,
                                  sim_um=sim_t_au.to("tau").units)
g_parameters_df = fill_parameters(g_parameters_df,
                                  name=afau_name,
                                  real_value=sim_af_au.magnitude,
                                  real_um=sim_af_au.units,
                                  sim_value=1,
                                  sim_um="afau")

In the following, we briefly discuss the derivation for each parameter.

> Note: some parameters' name might change from the manuscript to this Notebook

## Angiogenic factor (af) parameters

In our model we assume that the following partial differential equation (PDE) regulates the angiogenic factors (AFs) distribution ($af$):

$$
\begin{align}
\frac{\partial af}{\partial t} = D_{af}\nabla^2 af + V_{pT_{af}}\varphi (1 - shf(c)) - V_{uC_{af}}\cdot shf(c)af - V_{d_{af}} af
\end{align}
$$

Where:
- $shf$ is a smooth Heavyside function
- $D_{af}$ is the AFs diffusivity
- $V_{pT_{af}}$ is the AFs production rate
- $V_{uC_{af}}$ is the AFs uptake rate by the capillaries
- $V_{d_{af}}$ is the degradation rate.
- $\varphi$ is the tumor phase field

So we presume that $af$ is diffusing in the domain, while it is produced in the whole tumor body. Moreover, it decays in space and in capillaries.

In many cases, we considered the vascular endothelial growth factor (VEGF) as the main reference to estimate AFs parameters.

### Angiogenic factor diffusivity ($D_{af}$)
We could not find a specific experimental value for VEGF diffusivity, but its value has been estimated in several papers:

In [3]:
## Lai & Friedman (2019)
D_af_LaiFriedman = 8.64e-2 * ureg("cm^2 / day")  # value provided in paper
D_af_LaiFriedman = D_af_LaiFriedman.to("mm^2 / s")  # converted for comparison  [no experimental evidence]
print(f"Lai & Friedman (2019)  : {D_af_LaiFriedman:.2e}")

## Guerra et al. (2021)
D_af_Guerra = 1.16e-6 * ureg("mm^2 / s")  # value provided in paper   [no experimental evidence]
print(f"Guerra et al. (2021)   : {D_af_Guerra}")

## Phillips et al. (2020)
D_af_Phillips = 2.22e-6 * ureg("mm^2 / s")
print(f"Phillips et al. (2020) : {D_af_Phillips}")  # value provided in paper   [no experimental evidence]

Lai & Friedman (2019)  : 1.00e-04 millimeter ** 2 / second
Guerra et al. (2021)   : 1.16e-06 millimeter ** 2 / second
Phillips et al. (2020) : 2.22e-06 millimeter ** 2 / second


One of the oldest estimates is reported in Levine et al. (2001):

In [4]:
D_af_Levine = 1e-7 * ureg("mm^2 / s")
print(f"Levine et al. (2020)   : {D_af_Levine}")  # value provided in paper   [no experimental evidence]

Levine et al. (2020)   : 1e-07 millimeter ** 2 / second


This value was estimated using the [Stokes-Einstein equation of diffusivity](https://en.wikipedia.org/wiki/Einstein_relation_(kinetic_theory)) and exploiting the measures available on another protein (see Appendix B, section 3). This value (and thus, the same estimation) is the one used by Travasso et al. (2011). For all other values, we could not find a clear derivation path.

Thus, we also made an estimate for D_{af}, based on experimental evidence.

Kihara et al. (2013) measured the diffusivity of different peptides, with molecular weights ranging from 3 to 40 kDa. The diffusivity is measured in an actual extracellular matrix condensed by fibroblasts, making the experimental conditions closer to the retinal environment than a simple measure in water or serum.

In Table 1 they report the diffusion coefficients for two peptides with a mass close to VEGF:
- Alexa488 dextran (which is 10 kDa)
- and for FITC dextran (40 kDa)

In [5]:
D_Alexa488_dextran_serum = 81.7 * ureg("(um^2) / s")
D_FITC_dextran_serum = 44.7e-6 * ureg("(um^2) / s")

In Figure 8 they show that the diffusivity reduction in the extracellular matrix (ECM) compared to serum is quite small (27%), even though statistically significant.

In [6]:
D_Alexa488_dextran_matrix = 0.73 * D_Alexa488_dextran_serum
D_FITC_dextran_matrix = 0.73 * D_FITC_dextran_serum

We emplyed these measure to estimate VEGF diffusivity in retina by linear interpolation between the two values reported above.

According to the Uniprot entry for VEGFA (Uiprot ID: P15692) the VEGF isoforms have a molecular weight between 15,981 Da (Isoform VEGF111; identifier: P15692-10) and 45,467 Da (Isoform L-VEGF206; identifier: P15692-14). So we have:

In [7]:
# estimate as a line D = m * mol_w + q
m = (D_FITC_dextran_serum - D_Alexa488_dextran_serum) / (40e3 - 10e3)
q = D_Alexa488_dextran_serum - (10e3 * m)

# max diffusivity: smaller isoform in serum:
min_mol_w = 16e3  # Da
D_VEGF_max = (m * min_mol_w) + q
D_VEGF_max = D_VEGF_max.to("(mm^2) / s")  # convert to compare with others

# min diffusivity: bigger isoform in serum
max_mol_w = 45e3  # Da
D_VEGF_min = 0.73 * (m * max_mol_w) + q
D_VEGF_min = D_VEGF_min.to("(mm^2) / s")  # convert to compare with others

print(f"VEGF Diffusivity is in range: [{D_VEGF_min:.2g}, {D_VEGF_max:.2g}] millimiters^2 / seconds")

VEGF Diffusivity is in range: [1.9e-05 millimeter ** 2 / second, 6.5e-05 millimeter ** 2 / second] millimiters^2 / seconds


The range we found agrees with the values provided by the other authors.

In [8]:
D_af = (D_VEGF_min + D_VEGF_max) / 2
print("## D_af \n"
      f"value:  {D_af} \n"
      f"range:  [{D_af_Levine}, {D_af_LaiFriedman}]")

# add it to dictionary
g_parameters_df = fill_parameters(g_parameters_df,
                                  name="D_af",
                                  info="Diffusivity of angiogenic factor",
                                  real_value=D_af.magnitude,
                                  real_range_max=D_af_LaiFriedman.magnitude,
                                  real_range_min=D_af_Levine.magnitude,
                                  real_um=D_af.units,
                                  references="Estimated from Kihara, T., Ito, J., & Miyake, J. (2013). Measurement of Biomolecular Diffusion in Extracellular Matrix Condensed by Fibroblasts Using Fluorescence Correlation Spectroscopy. PLoS ONE, 8(11), 82382. https://doi.org/10.1371/JOURNAL.PONE.0082382",
                                  sim_value=D_af.to("sau^2 / tau").magnitude,
                                  sim_range_max=D_af_LaiFriedman.to("sau^2 / tau").magnitude,
                                  sim_range_min=D_af_Levine.to("sau^2 / tau").magnitude,
                                  sim_um=D_af.to("sau^2 / tau").units)

## D_af 
value:  4.2415938159916675e-05 millimeter ** 2 / second 
range:  [1e-07 millimeter ** 2 / second, 0.00010000000000000002 millimeter ** 2 / second]


### Angiogenic factor degradation rate ($V_{d_{af}}$)

According to Vempati et al. (2014), the degradation rate of VEGF in cellular conditions is:

In [9]:
## Values reported by Vempati et al. (2014)
V_d_af_max = 2.8e-4 * ureg("1 / s")
V_d_af_max = V_d_af_max.to("1 / hr")
V_d_af_min = 2.3e-4 * ureg("1 / s")
V_d_af_min = V_d_af_min.to("1 / hr")

# Take the average
V_d_af = (V_d_af_max + V_d_af_min) / 2

print(f"## V_d_af \n"
      f"value:  {V_d_af} \n"
      f"range:  [{V_d_af_min}, {V_d_af_max}]")

## V_d_af 
value:  0.918 / hour 
range:  [0.8280000000000001 / hour, 1.008 / hour]


In [10]:
# add V_d_af to parameters
g_parameters_df = fill_parameters(g_parameters_df,
                                  name="V_d_af",
                                  info="Angiogenic factor degradation rate",
                                  real_value=V_d_af.magnitude,
                                  real_range_max=V_d_af_max.magnitude,
                                  real_range_min=V_d_af_min.magnitude,
                                  real_um=V_d_af.units,
                                  references="Vempati, P., Popel, A. S., & Gabhann, F. Mac. (2014). Extracellular regulation of VEGF: isoforms, proteolysis, and vascular patterning. Cytokine Growth Factor Rev, 25(1), 1-19. https://doi.org/10.1016/j.cytogfr.2013.11.002",
                                  sim_value=V_d_af.to("1 / tau").magnitude,
                                  sim_range_max=V_d_af_max.to("1 / tau").magnitude,
                                  sim_range_min=V_d_af_min.to("1 / tau").magnitude,
                                  sim_um=V_d_af.to("1 / tau").units)

### Angiogenic factor production rate ($V_{pT_{af}}$)
Finley et al. (2013) have provided experimental measures and estimation of VEGF secretion rates:

In [11]:
p_VEGF_in_vitro_max = 2.65  # molecules / (cell * s)
p_VEGF_in_vitro_min = 0.03  # molecules / (cell * s)
p_VEGF_tumor_max = 0.023  # molecules / (cell * s)
p_VEGF_tumor_min = 0.007  # molecules / (cell * s)

To convert these values in the units required by our model $[\frac{afau}{tau}]$, we need to convert it making assumptions on:
- The weight of 1 molecule of VEGF ($W_{VEGF}$; unit of measure: grams)
- The cellular density in the tumor volume ($CD_{T}$; unit of measure: cells / mm^3)

#### $W_{VEGF}$
The weight of one molecule of VEGF can be computed easily from the molar mass:

In [12]:
# Avogadro's constant
c_Avogadro = 6.022e23

# Estimate W_VEGF
W_VEGF_min = (min_mol_w / c_Avogadro) * ureg("g")
W_VEGF_max = (max_mol_w / c_Avogadro) * ureg("g")
print(f"W_VEGF: [{W_VEGF_min}, {W_VEGF_max}]")

W_VEGF: [2.6569246097641978e-20 gram, 7.472600464961807e-20 gram]


#### $CD_T$
Assuming a cell radius in the order of:

In [13]:
R_c = 10 * ureg("um")

We can assume that where $\phi = 1$ all the tumor space is filled by cells, and we would have:

In [14]:
CD_T_max = 1 / ((4 / 3) * math.pi * (R_c ** 3))
CD_T_max = CD_T_max.to("1 / mL")

To be more conservative, we can assume that some of the tumor space contains ECM, and thus reduce this value of some percentage:

In [15]:
CD_T_min = CD_T_max * 0.75  # based on rigid packing hypothesis
print(f"CD_T: [{CD_T_min}, {CD_T_max}]")

CD_T: [179049310.97838232 / milliliter, 238732414.6378431 / milliliter]


#### Conversion
Gathering all together, we have that $V_{pT_{af}}$ can be in the order to:

In [16]:
V_pT_af_max = (p_VEGF_in_vitro_max * ureg("1 / s") * W_VEGF_max * CD_T_max)
V_pT_af_max = V_pT_af_max.to_compact().to("pg / (mL * s)")
V_pT_af_min = (p_VEGF_tumor_min * ureg("1 / s") * W_VEGF_min * CD_T_min)
V_pT_af_min = V_pT_af_min.to_compact().to("pg /(mL * s)")

V_pT_af = V_pT_af_max

print(f"## V_pT_af \n"
      f"value: {V_pT_af} \n"
      f"range: [{V_pT_af_min}, {V_pT_af_max}]")

## V_pT_af 
value: 47.27472674454134 picogram / milliliter / second 
range: [0.03330043644898509 picogram / milliliter / second, 47.27472674454134 picogram / milliliter / second]


In [17]:
# add it to parameters dict
g_parameters_df = fill_parameters(g_parameters_df,
                                  name="V_pT_af",
                                  info="Angiogenic factor production rate inside the tumor",
                                  real_value=V_pT_af.magnitude,
                                  real_range_max=V_pT_af_max.magnitude,
                                  real_range_min=V_pT_af_min.magnitude,
                                  real_um=V_d_af.units,
                                  sim_value=V_pT_af.to("afau / tau").magnitude,
                                  sim_range_min=V_pT_af_min.to("afau / tau").magnitude,
                                  sim_range_max=V_pT_af_max.to("afau / tau").magnitude,
                                  sim_um="afau / tau",
                                  references="Estimated from Finley, S. D., Dhar, M., & Popel, A. S. (2013). Compartment model predicts VEGF secretion and investigates the effects of VEGF Trap in tumor-bearing mice. Frontiers in Oncology, 3 JUL, 196. https://doi.org/10.3389/fonc.2013.00196")

### Angiogenic factor uptake in capillaries ($V_{uC_{af}}$)

In the original model, Travasso et al. (2011) estimated:

In [18]:
# estimate V_uc_af
V_uC_af = 0.004 * ureg("1 / s")
V_uC_af = V_uC_af.to("1 / hr")

print(f"## V_uC_af \n"
      f"value: {V_uC_af} \n")

## V_uC_af 
value: 14.4 / hour 


However, this value is not based on experimental evidence. Thus, we consider a range based on the degradation rate and on a value leading to no tip cells' activation

In [19]:
V_uC_af_min = 2 * V_d_af_min
V_uC_af = 10 * V_uC_af
V_uC_af_max = 10 * V_uC_af
# add it to parameters dict
g_parameters_df = fill_parameters(g_parameters_df,
                                  name="V_uc_af",
                                  info="Angiogenic factor uptake in capillaries",
                                  real_value=V_uC_af.magnitude,
                                  real_range_min=V_uC_af_min,
                                  real_range_max=V_uC_af_max,
                                  real_um=V_uC_af.units,
                                  references="Estimated by Travasso, R. D. M., Poire, E. C., Castro, M., Rodrguez-Manzaneque, J. C., & Hernandez-Machado, A. (2011). Tumor angiogenesis and vascular patterning: A mathematical model. PLoS ONE, 6(5), e19989. https://doi.org/10.1371/journal.pone.0019989",
                                  sim_value=V_uC_af.to("1 / tau").magnitude,
                                  sim_um="1 / tau",
                                  sim_range_min=V_uC_af_min.to("1 / tau"),
                                  sim_range_max=V_uC_af_max.to("1 / tau"))

/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/cast.py:1784: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result[:] = values


## Angiogenesis parameters

### Cell radius ($R_c$)

For the cell radius we assumed half of the maximum value provided by Bionumbers (Milo et al., 2010) at the ID 100432, which is:

In [20]:
print(f"R_c: {R_c}")

# add it to parameters dict
g_parameters_df = fill_parameters(g_parameters_df,
                                  name="R_c",
                                  info="Cell radius",
                                  real_value=R_c.magnitude,
                                  real_um=R_c.units,
                                  references="Milo et al. Nucl. Acids Res. (2010) 38 (suppl 1): D750-D753.; ID: 100432",
                                  sim_value=R_c.to("sau").magnitude,
                                  sim_um=R_c.to("sau").units)

R_c: 10 micrometer


This value also agrees with the endothelial cells dimension which can be inferred from the pictures reported by Shamloo, A. et al. (2008) (see Fig. 5A and 5B).

### Interface width of the capillaries ($\epsilon$)

The capillaries phase field model depends on a parameters $\epsilon$, regulating the interfacial width between the capillary and the surrounding tissue. We use the same value used by Travasso et al. (2011):

In [21]:
epsilon = (1.25 ** 2) * (ureg.um ** 2)  # notice: in Travasso provided as 'um' but it is probably an error

epsilon_real = epsilon
epsilon_sim = epsilon.to('sau ** 2')
print(f"epsilon: {epsilon_real:.2g}\n"
      f"         {epsilon_sim:.2g}")

epsilon: 1.6 micrometer ** 2
         2.4e-06 SpaceArbitraryUnit ** 2


In [22]:
# add it to parameters
g_parameters_df = fill_parameters(g_parameters_df,
                                  name="epsilon",
                                  info="Interface width",
                                  real_value=epsilon_real.magnitude,
                                  real_um=epsilon_real.units,
                                  references="Same provied by Travasso, R. D. M., Poire, E. C., Castro, M., Rodrguez-Manzaneque, J. C., & Hernandez-Machado, A. (2011). Tumor angiogenesis and vascular patterning: A mathematical model. PLoS ONE, 6(5), e19989. https://doi.org/10.1371/journal.pone.0019989",
                                  sim_value=epsilon_sim.magnitude,
                                  sim_um=epsilon_sim.units)

### Time step ($dt$)
The simulation time step is a parameter of the angiogenesis model because it regulates the frequency of tip cell activation. Indeed, at each time step maximum one tip cell can activate. We assume $dt$ to be the same as the arbitrary time unit:

In [23]:
dt = sim_t_au
print(f"dt: {dt}")

dt: 26 minute


This makes sure that the activation of new tip cells does not lead to discontinuous capillaries, because the maximum cell displacement (i.e. the maximum tip cells' velocity times $dt$) is lower than a TC radius (see section regarding $G_M$)

In [24]:
g_parameters_df = fill_parameters(g_parameters_df,
                                  name="dt",
                                  info="Simulation time step",
                                  real_value=dt.magnitude,
                                  real_um=dt.units,
                                  references="Same provied by Travasso, R. D. M., Poire, E. C., Castro, M., Rodrguez-Manzaneque, J. C., & Hernandez-Machado, A. (2011). Tumor angiogenesis and vascular patterning: A mathematical model. PLoS ONE, 6(5), e19989. https://doi.org/10.1371/journal.pone.0019989",
                                  sim_value=dt.to("tau").magnitude,
                                  sim_um=dt.to("tau").units)

### Angiogenic factor value for maximum proliferation ($T_p$)

Based on the measures realized by Tracqui P. et al. (2005), Travasso et al. (2011) estimated a linear increase of the proliferation of endothelial cells only up to a certain angiogenic factor level, $T_p$. This $T_p$ value is set to:

In [25]:
T_p = 0.3 * ureg.afau

This value allowed them to obtain results comparable with angiogenic experiments. Thus, we kept the same.

In [26]:
T_p_real = T_p.to("pg / mL")
T_p_sim = T_p

g_parameters_df = fill_parameters(g_parameters_df,
                                  name="T_p",
                                  info="Angiogenic factor concentration for maximum proliferation",
                                  real_value=T_p_real.magnitude,
                                  real_um=T_p_real.units,
                                  references="None",
                                  sim_value=T_p_sim.magnitude,
                                  sim_um=T_p_sim.units)

### Endothelial cells proliferation rate ($\alpha_p$)

Based on the measures by Tracqui, P. (2005), Travasso et al. (2011) estimated a normal value for the maximum proliferation rate of endothelial cells to be:


In [27]:
endothelial_cells_proliferation_rate = 0.97 * (1 / ureg.hr)

Pairing this value to the *in silico* maximum proliferation rate, which is $\alpha_p T_p$, we get:

In [28]:
alpha_p = endothelial_cells_proliferation_rate.to("1 / tau") / T_p
alpha_p = alpha_p.to_compact()
print(f"alpha_p: {alpha_p:.2g}")

alpha_p: 1.4 / AFsArbitraryUnit / TimeArbitraryUnit


Which is in agreement with the value used by Vilanova et al. (2016).

In [29]:
alpha_p_real = alpha_p.to("(mL / pg) * (1 / hr)")
alpha_p_sim = alpha_p

In [30]:
g_parameters_df = fill_parameters(g_parameters_df,
                                  name="alpha_p",
                                  info="Endothelial cells proliferation rate",
                                  real_value=alpha_p_real.magnitude,
                                  real_um=alpha_p_real.units,
                                  sim_value=alpha_p_sim.magnitude,
                                  sim_um=alpha_p_sim.units)

### Minimum angiogenic factor concentration for tip cells activation ($T_c$)

Despite difficult to measure, a threshold in AF concentration for tip cell activation is likely to exist, since they need to bind to cellular surface receptors to influence cells behaviour. Considering VEGF, it has been measured that VEGF receptors (VEGFR) have a density of (Imoukhuede, P. I. et al. (2011)):

In [31]:
d_VEGFR_cell_min = 1000  # VEGFR molecules / cell
d_VEGFR_cell_max = 10000 # VEGFR molecules / cell

Considering the cell density we have assumed above, this leads to the following concentration:

In [32]:
c_VEGFR_cell_min = (d_VEGFR_cell_min * CD_T_min) / c_Avogadro
c_VEGFR_cell_max = (d_VEGFR_cell_max * CD_T_max) / c_Avogadro
print(f"VEGFR concentration: [{c_VEGFR_cell_min}, {c_VEGFR_cell_max}]  unit: moles / mL")

VEGFR concentration: [2.9732532543736684e-13 / milliliter, 3.964337672498225e-12 / milliliter]  unit: moles / mL


Assuming that the molar concentration of VEGF should be in the same order of magnitude, we can assume T_c to be in the order of:

In [33]:
T_c_min = 1e-13 * min_mol_w * ureg("g / mL")
T_c_min = T_c_min.to("afau")
T_c_max = 1e-13 * max_mol_w * ureg("g / mL")
T_c_max = T_c_max.to("afau")
print(f"T_c: [{T_c_min}, {T_c_max}]")
T_c_sim = 0.5 * ureg("afau")

T_c: [0.26666666666666666 AFsArbitraryUnit, 0.7499999999999999 AFsArbitraryUnit]


However, nothing guarantee that the minimum concentration can also be lower. Thus, we assume:

In [34]:
T_c_sim_min = T_c_sim / 10
T_c_real_min = T_c_sim_min.to("pg / mL")  # eventually, not used in the paper
T_c_real = T_c_sim.to("pg / mL")

print(f"T_c: \n"
      f"\t sim: {T_c_real} (real) {T_c_sim} (sim) \n"
      f"\t min: {T_c_real_min} (real) {T_c_sim_min} (sim) \n")

g_parameters_df = fill_parameters(g_parameters_df,
                                  name="T_c",
                                  info="Minimum af for Tip Cells Activation",
                                  real_value=T_c_real.magnitude,
                                  real_um=T_c_real.units,
                                  real_range_min=T_c_real_min.magnitude,
                                  sim_value=T_c_sim.magnitude,
                                  sim_um=T_c_sim.units,
                                  sim_range_min=T_c_sim_min.magnitude,
                                  references="Imoukhuede, P. I., & Popel, A. S. (2011). Quantification and cell-to-cell variation of vascular endothelial growth factor receptors. Experimental Cell Research, 317(7), 955-965. https://doi.org/10.1016/J.YEXCR.2010.12.014")

T_c: 
	 sim: 3000.0000000000005 picogram / milliliter (real) 0.5 AFsArbitraryUnit (sim) 
	 min: 300.00000000000006 picogram / milliliter (real) 0.05 AFsArbitraryUnit (sim) 


### Mimimum angiogenic factor gradient for Tip Cells activation ($G_m$)
From Shamloo et al (2008) we have:

In [35]:
G_m_real = 14 * ureg("ng / (mL * mm)")
# converting to arbitrary units:
G_m_sim = G_m_real.to("afau / sau")
print(f"## G_m\n"
      f"real: {G_m_real}\n"
      f"sim : {G_m_sim}")

## G_m
real: 14.0 nanogram / milliliter / millimeter
sim : 1.8666666666666667 AFsArbitraryUnit / SpaceArbitraryUnit


In [36]:
g_parameters_df = fill_parameters(g_parameters_df,
                                  name="G_m",
                                  info="Minimum af gradient required for Tip Cells Activation",
                                  real_value=G_m_real.magnitude,
                                  real_um=G_m_real.units,
                                  sim_value=G_m_sim.magnitude,
                                  sim_um=G_m_sim.units,
                                  references="Shamloo, A., Ma, N., Poo, M. M., Sohn, L. L., & Heilshorn, S. C. (2008). Endothelial cell polarization and chemotaxis in a microfluidic device. Lab on a Chip, 8(8), 1292-1299. https://doi.org/10.1039/B719788H")

### Minimum value for capillaries field c to allow Tip Cells activation ($\varphi_{th}$)

Tip cells can originate only inside the already formed capillaries. Thus, we set:

In [37]:
phi_th = 0.9

And have:

In [38]:
g_parameters_df = fill_parameters(g_parameters_df,
                                  name="phi_th",
                                  info="Minimum capillaries field value for Tip Cells activation",
                                  sim_value=phi_th)

### Gradient for maximum tip cells velocity ($G_M$)
According to Travasso et al. (2011):

In [39]:
G_M_real = 3 * G_m_real
G_M_sim = 3 * G_m_sim

print(f"G_M: \n"
      f"     (real) {G_M_real:.2g}\n"
      f"     (sim)  {G_M_sim:.2g}")

G_M: 
     (real) 42 nanogram / milliliter / millimeter
     (sim)  5.6 AFsArbitraryUnit / SpaceArbitraryUnit


This value is estimated by Stokes et al. (1991):

In [40]:
g_parameters_df = fill_parameters(g_parameters_df,
                                  name="G_M",
                                  info="af gradient for maximum tip cells velocity",
                                  real_value=G_M_real.magnitude,
                                  real_um=G_M_real.units,
                                  sim_value=G_M_sim.magnitude,
                                  sim_um=G_M_sim.units)

### Extreme values for capillaries field ($\varphi_{min}$ and $\varphi_{max}$)

For the system construction, these values are just 1 and -1:

In [41]:
phi_min = -1  # define minimum phi value

# add it to parameters
g_parameters_df = fill_parameters(g_parameters_df,
                                  name="phi_min",
                                  info="Min capillaries field value",
                                  sim_value=phi_min)

phi_max = 1  # define maximum phi value

# add it to parameters
g_parameters_df = fill_parameters(g_parameters_df,
                                  name="phi_max",
                                  info="Max capillaries field value",
                                  sim_value=phi_max)

### Motility value for capillaries phase field ($M$)

Travasso et al. assume this value to be $10^{-15} [\frac{m^2}{s}]$. We assume the same:

In [42]:
M_real = 1e-15 * ureg("m ** 2 / s")
M_real = M_real.to("mm ** 2 / s")
M_sim = M_real.to("sau ** 2 / tau")

g_parameters_df = fill_parameters(g_parameters_df,
                                  name="M",
                                  info="'Motility' for the capillaries phase field",
                                  real_value=M_real.magnitude,
                                  real_um=M_real.units,
                                  sim_value=M_sim.magnitude,
                                  sim_um=M_sim.units)

### Chemotactic coefficient for tip cells ($\chi$)

Travasso assumes that the maximum tip cells velocity is in the order of (Figure 1; Travasso et al. 2011):

In [43]:
max_tip_cells_velocity_real = 0.35 * (ureg.um / ureg.min)
print(f"max tip cells velocity (real): {max_tip_cells_velocity_real:.2g}")
max_tip_cells_velocity_sim = max_tip_cells_velocity_real.to("sau / tau")
print(f"max tip cells velocity (sim): {max_tip_cells_velocity_sim:.2g}")

max tip cells velocity (real): 0.35 micrometer / minute
max tip cells velocity (sim): 0.011 SpaceArbitraryUnit / TimeArbitraryUnit


And recent experiments confirm this result. Blinder Y.J. et al. (2015) report an average tip cell velocity of $17 \frac{\mu m}{hr} =  0.28 \frac{\mu m}{min}$.

This leads to a $\chi$ value of:

In [44]:
chi_sim = max_tip_cells_velocity_sim / G_M_sim
print(f"chi (sim): {chi_sim:.2g}")
chi_real = max_tip_cells_velocity_real / G_M_real
print(f"chi (real): {chi_real:.2g}")

chi (sim): 0.002 SpaceArbitraryUnit ** 2 / AFsArbitraryUnit / TimeArbitraryUnit
chi (real): 0.0083 micrometer * milliliter * millimeter / minute / nanogram


In [45]:
g_parameters_df = fill_parameters(g_parameters_df,
                                  name="chi",
                                  info="chemotactic constant for tip cells",
                                  real_value=chi_real.magnitude,
                                  real_um=chi_real.units,
                                  sim_value=chi_sim.magnitude,
                                  sim_um=chi_sim.units)

### Minimum Tip Cells distance
The generation of new Tip Cells is inhibited by other Tip Cells due to the Notch Pathway. This mechanism prevents the formation of two neighbor tip cells, thus, Travasso et al. (2011) assumed that exists a minimum distance for the activation of two tip cells equal to 4 cell radiuses:

In [46]:
min_tipcell_distance_real = 4 * R_c
min_tipcell_distance_sim = min_tipcell_distance_real.to("sau")

print(f"min tipcell distance: \n"
      f"                      (real) {min_tipcell_distance_real:.2g}\n"
      f"                      (sim)  {min_tipcell_distance_sim:.2g}")

min tipcell distance: 
                      (real) 40 micrometer
                      (sim)  0.05 SpaceArbitraryUnit


In [47]:
g_parameters_df = fill_parameters(g_parameters_df,
                                  name="min_tipcell_distance",
                                  info="Min Tip Cells distance (due to Notch pathway)",
                                  real_value=min_tipcell_distance_real.magnitude,
                                  real_um=min_tipcell_distance_real.units,
                                  sim_value=min_tipcell_distance_sim.magnitude,
                                  sim_um=min_tipcell_distance_sim.units)

## Tumor parameters

### Initial tumor dimension factor
The dimension of each tumor is reported in the `input_patients_data/patients_parameters.json` file. With the initial tumor dimension factor, the user can control the initial dimension of the tumor as a fraction of the initial dimension. For instance, an initial tumor dimension ratio of 0.5, will make the simulation start with all the axes of the tumor equal to the 50% of their actual dimension.


In [48]:
initial_tumor_dimension_factor = 1

g_parameters_df = fill_parameters(g_parameters_df,
                                  name="tdf_i",
                                  info="Initial Tumor Dimension factor",
                                  real_value=initial_tumor_dimension_factor,
                                  real_um="None",
                                  sim_value=initial_tumor_dimension_factor,
                                  sim_um="None",
                                  references="None")

### Tumor Growth Rate
The average growth for CNS hemangioblastoma related to VHL (we could not find data for Retinal Hemangioblastoma) has been reported to be in the order of 0.35% per year. This means that, given an initial volume $V_i$, the volume evolution in time can be described by $V(t) = V_i \cdot 1.35^{t}$, where t is in [years]. (Ye D. et al.; 2012)

Given that 1 year correspond to:

In [49]:
year_in_tau = (1 * ureg("year")).to("tau")
print(year_in_tau)

20229.23076923077 TimeArbitraryUnit


The growth rate in terms of tau is $1.3^{\frac{1}{year_in_tau}}$:

In [50]:
tgr = 1.35 ** (1 / year_in_tau.magnitude)
tgr_max = 1.5 ** (1 / year_in_tau.magnitude)
print(tgr)
print(tgr_max)

1.0000148353055012
1.0000200437266367


In [51]:
g_parameters_df = fill_parameters(g_parameters_df,
                                  name="tgr",
                                  info="Tumor growth rate",
                                  real_value=1.35,
                                  real_range_max=1.5,
                                  real_um="volume fraction per year",
                                  sim_value=tgr,
                                  sim_range_max=tgr_max,
                                  sim_um="volume fraction per tau",
                                  references="Estimated using Ye D. et al. (2012)")

# Complete parameters
The complete parameters table is:

In [52]:
from IPython.display import display

g_parameters_df.set_index("name", inplace=True)  # needed for simulation

display(g_parameters_df)

,info,real_value,real_range_max,real_range_min,real_um,sim_value,sim_range_max,sim_range_min,sim_um,references,influence
name,,,,,,,,,,,
SpaceArbitraryUnit,None,800,NaN,NaN,micrometer,1.000000,NaN,NaN,SpaceArbitraryUnit,None,None
TimeArbitraryUnit,None,26,NaN,NaN,minute,1.000000,NaN,NaN,TimeArbitraryUnit,None,None
AFsArbitraryUnit,None,6000.0,NaN,NaN,picogram / milliliter,1.000000,NaN,NaN,afau,None,None
D_af,Diffusivity of angiogenic factor,0.000042,0.0001,0.0,millimeter ** 2 / second,0.103389,0.24375,0.000244,SpaceArbitraryUnit ** 2 / TimeArbitraryUnit,"Estimated from Kihara, T., Ito, J., & Miyake, ...",None
V_d_af,Angiogenic factor degradation rate,0.918,1.008,0.828,1 / hour,0.397800,0.4368,0.3588,1 / TimeArbitraryUnit,"Vempati, P., Popel, A. S., & Gabhann, F. Mac. ...",None
V_pT_af,Angiogenic factor production rate inside the t...,47.274727,47.274727,0.0333,1 / hour,12.291429,12.291429,0.008658,afau / tau,"Estimated from Finley, S. D., Dhar, M., & Pope...",None
V_uc_af,Angiogenic factor uptake in capillaries,144.0,1440.0 / hour,1.6560000000000001 / hour,1 / hour,62.400000,623.9999999999999 / TimeArbitraryUnit,0.7176 / TimeArbitraryUnit,1 / tau,"Estimated by Travasso, R. D. M., Poire, E. C.,...",None
R_c,Cell radius,10,None,None,micrometer,0.012500,None,None,SpaceArbitraryUnit,Milo et al. Nucl. Acids Res. (2010) 38 (suppl ...,None
epsilon,Interface width,1.5625,None,None,micrometer ** 2,0.000002,None,None,SpaceArbitraryUnit ** 2,"Same provied by Travasso, R. D. M., Poire, E. ...",None


So we can convert it to csv:

In [53]:
g_parameters_df.to_csv("out/g_parameters.csv")

# References

Lai, X., & Friedman, A. (2019). Mathematical modeling in scheduling cancer treatment with combination of VEGF inhibitor and chemotherapy drugs. Journal of Theoretical Biology, 462, 490-498. https://doi.org/10.1016/J.JTBI.2018.11.018

Liao, K.L. , Bai, X.F. , Friedman, A. , 2014. Mathematical modeling of interleukin-27 induction of anti-tumor t cells response. PLoS One 9 (3), e91844.

Guerra, A., Belinha, J., Mangir, N., MacNeil, S., & Natal Jorge, R. (2021). Sprouting Angiogenesis: A Numerical Approach with Experimental Validation. Annals of Biomedical Engineering, 49(2), 871-884. https://doi.org/10.1007/S10439-020-02622-W/TABLES/1

Phillips, C. M., Lima, E. A. B. F., Woodall, R. T., Brock, A., & Yankeelov, T. E. (2020). A hybrid model of tumor growth and angiogenesis: In silico experiments. PLOS ONE, 15(4), e0231137. https://doi.org/10.1371/JOURNAL.PONE.0231137

Levine, H. A., Pamuk, S., Bölümü, M., Sleeman, B. D., Nilsen-Hamilton, M., Cotran, R. A. S., Leapman, S. B., Folkman, J., & Levine, H. A. (2001). Mathematical modeling of capillary formation and development in tumor angiogenesis: Penetration into the stroma. Bulletin of Mathematical Biology 2001 63:5, 63(5), 801-863. https://doi.org/10.1006/BULM.2001.0240

Kihara, T., Ito, J., & Miyake, J. (2013). Measurement of Biomolecular Diffusion in Extracellular Matrix Condensed by Fibroblasts Using Fluorescence Correlation Spectroscopy. PLoS ONE, 8(11), 82382. https://doi.org/10.1371/JOURNAL.PONE.0082382

Kut, C., Mac Gabhann, F., & Popel, A. S. (2007). Where is VEGF in the body? A meta-analysis of VEGF distribution in cancer. British Journal of Cancer 2007 97:7, 97(7), 978-985. https://doi.org/10.1038/sj.bjc.6603923

Milo et al. Nucl. Acids Res. (2010) 38 (suppl 1): D750-D753.

Tracqui, P., Liu, J. W., Collin, O., Clement-Lacroix, J., & Planus, E. (2005). Global analysis of endothelial cell line proliferation patterns based on nutrient-depletion models: implications for a standardization of cell proliferation assays. Cell Proliferation, 38(3), 119-135. https://doi.org/10.1111/J.1365-2184.2005.00337.X

Los, M., Aarsman, C. J. M., Terpstra, L., Wittebol-Post, D., Lips, C. J. M., Blijham, G. H., & Voest, E. E. (1997). Elevated ocular levels of vascular endothelial growth factor in patients with von Hippel-Lindau disease. Annals of Oncology, 8(10), 1015-1022. https://doi.org/10.1023/A:1008213320642

Karayiannakis, A. J., Zbar, A., Polychronidis, A., & Simopoulos, C. (2003). Serum and Drainage Fluid Vascular Endothelial Growth Factor Levels in Early Surgical Wounds. European Surgical Research, 35(6), 492-496. https://doi.org/10.1159/000073388

Shamloo, A., Ma, N., Poo, M. M., Sohn, L. L., & Heilshorn, S. C. (2008). Endothelial cell polarization and chemotaxis in a microfluidic device. Lab on a Chip, 8(8), 1292-1299. https://doi.org/10.1039/B719788H

Stokes, C. L., Lauffenburger, D. A., & Williams, S. K. (1991). Migration of individual microvessel endothelial cells: stochastic model and parameter measurement.

Finley, S. D., Dhar, M., & Popel, A. S. (2013). Compartment model predicts VEGF secretion and investigates the effects of VEGF Trap in tumor-bearing mice. Frontiers in Oncology, 3 JUL, 196. https://doi.org/10.3389/fonc.2013.00196

Na, X., Wu, G., Ryan, C. K., Schoen, S. R., Di’Santagnese, P. A., & Messing, E. M. (2003). Overproduction of vascular endothelial growth factor related to von Hippel-Lindau tumor suppressor gene mutations and hypoxia-inducible factor-1α expression in renal cell carcinomas. Journal of Urology, 170(2 I), 588-592. https://doi.org/10.1097/01.ju.0000074870.54671.98

Blinder, Y. J., Freiman, A., Raindel, N., Mooney, D. J., & Levenberg, S. (2015). Vasculogenic dynamics in 3D engineered tissue constructs. Scientific Reports, 5(1), 1-8. https://doi.org/10.1038/srep17840

Goswami, A., Surve, A., & Venkatesh, P. (2021). Optical Coherence Tomography Angiography of Early Stage 1a Retinal Hemangioblastoma in Von-Hippel-Lindau. Journal of Kidney Cancer and VHL, 8(3), 15-18. https://doi.org/10.15586/JKCVHL.V8I3.158

Arora, S., Chung, H., & Damji, K. (2015). Ophthalmology and Clinical Research Optic Disk Size Assessment Techniques: Photo Essay. Int J Ophthalmol Clin Res, 2, 1.

Ye, D. Y., Bakhtian, K. D., Asthagiri, A. R., & Lonser, R. R. (2012). Effect of pregnancy on hemangioblastoma development and progression in von Hippel-Lindau disease: Clinical article. Journal of Neurosurgery, 117(5), 818-824. https://doi.org/10.3171/2012.7.JNS12367

Imoukhuede, P. I., & Popel, A. S. (2011). Quantification and cell-to-cell variation of vascular endothelial growth factor receptors. Experimental Cell Research, 317(7), 955-965. https://doi.org/10.1016/J.YEXCR.2010.12.014

Bateman, A., Martin, M. J., Orchard, S., Magrane, M., Agivetova, R., Ahmad, S., Alpi, E., Bowler-Barnett, E. H., Britto, R., Bursteinas, B., Bye-A-Jee, H., Coetzee, R., Cukura, A., da Silva, A., Denny, P., Dogan, T., Ebenezer, T. G., Fan, J., Castro, L. G., … Teodoro, D. (2021). UniProt: the universal protein knowledgebase in 2021. Nucleic Acids Research, 49(D1), D480-D489. https://doi.org/10.1093/NAR/GKAA1100